In [ ]:
import pandas as pd
import numpy as nm
from sklearn.model_selection import train_test_split as ttp
from sklearn.metrics import classification_report
import re
import string
import matplotlib.pyplot as plt



In [ ]:
data_true=pd.read_csv('/content/drive/MyDrive/fake news/true.csv')
data_fake=pd.read_csv('/content/drive/MyDrive/fake news/fake.csv')


In [ ]:
data_true.head(5)

In [ ]:
data_fake.head(5)

In [ ]:
data_true.shape,data_fake.shape

In [ ]:
data_true["class"]=1
data_fake["class"]=0

In [ ]:
data_true_mannual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
  data_true.drop([i],axis=0,inplace=True)



In [ ]:
data_fake_mannual_testing = data_fake.tail(10)
for i in range(21416,21406,-1):
  data_fake.drop([i],axis=0,inplace=True)

In [ ]:
data_mannual_testing = pd.concat([data_fake_mannual_testing,data_true_mannual_testing],axis=0)
data_mannual_testing.to_csv("manual_testing.csv")

In [ ]:
data_merge = pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

In [ ]:
print(data_merge.groupby(['subject'])['text'].count())
data_merge.groupby(['subject'])['text'].count().plot(kind="bar")
plt.title("Article per subject",size=20)
plt.xlabel("Category",size=20)
plt.ylabel("Article count",size=20)

In [ ]:
print(data_merge.groupby(['class'])['text'].count())
print("0 = Fake news\n1 = True news")
data_merge.groupby(['class'])['text'].count().plot(kind="pie")
plt.title("Fake news and True news",size=20)
plt.show()

In [ ]:
data = data_merge.drop(["title","subject","date"],axis=1)
data.head(10)

In [ ]:
data=data.sample(frac=1)
data.head(10)

In [ ]:
data.isnull().sum()

In [ ]:
def filtering(data):
  text = data.lower()
  text = re.sub('\[.*?\]','',text)
  text = re.sub("\\W"," ",text)
  text =re.sub('http?://\s+|www\.S+','',text)
  text = re.sub('<.*?>+','',text)
  text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text = re.sub('\w*\d\w*','',text)
  return text

In [ ]:
data["text"]=data["text"].apply(filtering)
data.head(10)

In [ ]:
x=data["text"]
y=data["class"]

In [ ]:
x_train,x_test,y_train,y_test=ttp(x,y,test_size=0.25,random_state=0)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer()
xv_train = vector.fit_transform(x_train)
xv_test = vector.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
LR.fit(xv_train,y_train)
accuracy = LR.score(xv_test,y_test)
print("Accuary is",accuracy)
pred_LR=LR.predict(xv_test)
print(classification_report(y_test,pred_LR))

In [ ]:
def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(filtering)
  new_x_test = new_def_test["text"]
  new_xv_test = vector.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  def output_label(n):
    if n == 0:
      return "Fake News"
    elif n == 1:
      return "True News"
  return print("\n\nLR Prediction: {}".format(output_label(pred_LR[0])))

In [ ]:
news = str(input())
manual_testing(news)